In [ ]:
import keras
from keras.layers import Conv2D,Input,Flatten,Layer,Dense,Reshape,Conv2DTranspose,Lambda
from keras.backend import random_normal,exp
import keras.backend as K
import numpy as np
import tensorflow as tf

In [ ]:
class Sample(Layer):
  # Reparameterization trick with epsilon instead of sampling from the prev layer output which blocks gradients to backpropagate
  def call(self,z_mean,z_var):

    global Dimension

    # epsilon : additional parameter which gives randomness to the latent variable while keeping the mean and variance in the computation graph
    batch_size =  tf.shape(z_mean)[0]
    epsilon = random_normal(shape=(batch_size,Dimension))

    return z_mean +  exp(0.5 * z_var) * epsilon


In [ ]:
class NamingLayer(Layer): # for loss function
   def call(self,X):  return X

def iden(_,X):  return X

In [ ]:
class VAE_Loss(Layer):
  def call(self,Y_true, Y_pred,Z_mean,Z_var):

    #recon_loss = tf.reduce_mean(K.sum(keras.losses.binary_crossentropy(Y_true, Y_pred), axis=(1, 2) ))
    recon_loss = tf.reduce_mean(K.sum(K.abs(Y_pred-Y_true),axis=(1,2)))
    KL_div =  -0.5 * (1 + Z_var - K.square(Z_mean) - K.exp(Z_var))
    KL_div = K.mean(K.sum(KL_div, axis=1))
    return recon_loss , KL_div


In [ ]:
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
print(mnist_digits.shape)

(70000, 28, 28, 1)


In [ ]:
Dimension = 4
batch_size = 128
inputs = Input(shape = (mnist_digits.shape[1:]))




# Encoder : Proxy Q(Z|X) to approximate true posterior : core idea of Variational inference

x = Conv2D(32, 3, activation="relu", strides=2, padding="same")(inputs)
x = Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = Flatten()(x)
x = Dense(16, activation="relu")(x)

Z_mean = Dense(Dimension,name="mean_distribution")(x)
Z_var  = Dense(Dimension,name="var_distribution")(x)


# Sampling Z from predicted distribution
Latent = Sample(name="latent_variable")(Z_mean, Z_var)



# Decoder : P(X|z) : the Model which gives X input z
x = Dense(16,name = "latent_input")(Latent)
x = Dense(7*7*16)(x)
x = Reshape((7,7,16))(x)
x = Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
y_pred = Conv2DTranspose(1, 3, activation="sigmoid", padding="same",name="Output")(x)


recon_loss , KL_div = VAE_Loss(name="vae_loss")(inputs, y_pred,Z_mean,Z_var)

recon_loss = NamingLayer(name = "recon")(recon_loss)
KL_div = NamingLayer(name = "KL_div")(KL_div)

VAE = keras.Model(inputs=inputs, outputs=[recon_loss , KL_div], name="VAE")
VAE.summary()


Model: "VAE"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 14, 14, 32)           320       ['input_3[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 7, 7, 64)             18496     ['conv2d_2[0][0]']            
                                                                                                  
 flatten_1 (Flatten)         (None, 3136)                 0         ['conv2d_3[0][0]']            
                                                                                                

In [ ]:
VAE.compile(keras.optimizers.Adam(), loss={'recon':iden , 'KL_div': iden})
VAE.fit(mnist_digits, mnist_digits, batch_size=batch_size, epochs=50)

Epoch 1/50
547/547 [==============================] - 8s 9ms/step - loss: 109.6200 - recon_loss: 108.3312 - KL_div_loss: 1.2889
Epoch 2/50
547/547 [==============================] - 5s 9ms/step - loss: 86.1427 - recon_loss: 82.6101 - KL_div_loss: 3.5326
Epoch 3/50
547/547 [==============================] - 5s 10ms/step - loss: 77.2995 - recon_loss: 71.4639 - KL_div_loss: 5.8357
Epoch 4/50
547/547 [==============================] - 5s 9ms/step - loss: 73.1983 - recon_loss: 66.6706 - KL_div_loss: 6.5278
Epoch 5/50
547/547 [==============================] - 5s 9ms/step - loss: 71.3372 - recon_loss: 64.6451 - KL_div_loss: 6.6920
Epoch 6/50
547/547 [==============================] - 5s 10ms/step - loss: 70.1501 - recon_loss: 63.3786 - KL_div_loss: 6.7716
Epoch 7/50
547/547 [==============================] - 5s 9ms/step - loss: 69.3486 - recon_loss: 62.5386 - KL_div_loss: 6.8100
Epoch 8/50
547/547 [==============================] - 5s 10ms/step - loss: 68.7331 - recon_loss: 61.9028 - KL_div_

In [ ]:
# INFERENCE MODEL
InferrenceModel = keras.Model(inputs=VAE.get_layer("latent_input").input, outputs=VAE.get_layer("Output").output, name="VAE")
proxy = keras.Model(inputs=VAE.input, outputs=[VAE.get_layer("mean_distribution").output,VAE.get_layer("var_distribution").output], name="proxy")


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
len(x_test)

10000

In [ ]:
test = x_test[1000]
cv2_imshow(test)
test = np.expand_dims(test, -1).astype("float32") / 255
latent_distribution = proxy( np.expand_dims(test,0))
print(latent_distribution)





[<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[nan, nan, nan, nan]], dtype=float32)>, <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[nan, nan, nan, nan]], dtype=float32)>]


In [ ]:
z_mean, z_var = latent_distribution


for i in range(10):
  latent =  z_mean +  exp(0.5 * z_var) *  random_normal(shape=(1,Dimension))
  print(latent)


  pred = InferrenceModel(latent).numpy()
  pred = pred.reshape(28,28)
  pred = (pred * 255).astype("int16")
  cv2_imshow(pred)

tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)


tf.Tensor([[nan nan nan nan]], shape=(1, 4), dtype=float32)
